In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import csv

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

folder_path = 'SeaData\\csv\\d10'
date = 'd10'

with open(f"Statistics for RNN {date}.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f'Statistics for RNN {date}'])
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        print(f'Processing file: {file_path}')
        tag = ''
        if 'dbiastg' in file_name:
            tag = 'dbiastg'
        else:
            tag = 'biastg'   
        writer.writerow([file_name])       

        data = pd.read_csv(file_path, index_col='date', parse_dates=True)

        # Prepare the data
        data = data[tag]
        data = data.asfreq('B').fillna(method='ffill')  # Fill missing values

        # Normalize the dataset
        scaler = MinMaxScaler(feature_range=(0, 1))
        data = scaler.fit_transform(data.values.reshape(-1, 1))

        # Split the data into training and testing sets
        train_size = int(len(data) * 0.8)
        test_size = len(data) - train_size
        train_data, test_data = data[0:train_size,:], data[train_size:len(data),:]

        # Reshape the data into X=t and Y=t+1
        look_back = 1
        X_train, Y_train = create_dataset(train_data, look_back)
        X_test, Y_test = create_dataset(test_data, look_back)

        # Reshape the input to be [samples, time steps, features]
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        # Create and fit the RNN network
        model = Sequential()
        model.add(SimpleRNN(4, input_shape=(1, look_back)))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

        # Make predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Invert predictions
        train_predict = scaler.inverse_transform(train_predict)
        Y_train = scaler.inverse_transform([Y_train])
        test_predict = scaler.inverse_transform(test_predict)
        Y_test = scaler.inverse_transform([Y_test])

        # Evaluate the model
        mae = mean_absolute_error(Y_test[0], test_predict[:,0])
        mse = mean_squared_error(Y_test[0], test_predict[:,0])
        rmse = np.sqrt(mse)
        mbe = np.mean(Y_test[0] - test_predict[:,0])
        mape = mean_absolute_percentage_error(Y_test[0], test_predict[:,0])


        print(f"RMSE: {rmse:.2f}")
        writer.writerow(['RMSE',rmse])
        print(f"MAE: {mae:.2f}")
        writer.writerow(['MAE',mae])
        print(f"MBE: {mbe:.2f}")
        writer.writerow(['MBE',mbe])
        print(f"MAPE: {mape:.2f}%")
        writer.writerow(['MAPE',mape])

        # Plot the results
        plt.figure(figsize=(14, 6))
        plt.plot(data, label='Observed')
        plt.plot(np.concatenate((train_predict, test_predict)),color='r', label='Predicted')
        plt.xlabel('Date')
        plt.ylabel(tag)
        plt.legend()
        plt.show()

        plt.figure(figsize=(14, 6))
        plt.plot(train_data, label='Train Data', color='blue')
        plt.plot(np.arange(len(train_data), len(train_data) + len(test_data)), test_data, label='Test Data', color='green')
        plt.plot(np.concatenate((train_predict, test_predict)), color='red', linestyle='--', label='Predicted')
        plt.xlabel('Date')
        plt.ylabel(tag)
        plt.legend()
        plt.show()


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import csv

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

folder_path = 'SeaData\\csv\\d10'
date = 'd10'

with open(f"Statistics for RNN {date}.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f'Statistics for RNN {date}'])

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        print(f'Processing file: {file_path}')
        tag = ''
        if 'dbiastg' in file_name:
            tag = 'dbiastg'
        else:
            tag = 'biastg'
        writer.writerow([file_name])

        data = pd.read_csv(file_path, index_col='date', parse_dates=True)

        # Prepare the data
        data = data[tag]
        data = data.asfreq('B').fillna(method='ffill')  # Fill missing values

        # Normalize the dataset
        scaler = MinMaxScaler(feature_range=(0, 1))
        data = scaler.fit_transform(data.values.reshape(-1, 1))

        # Split the data into training and testing sets
        train_size = int(len(data) * 0.8)
        test_size = len(data) - train_size
        train_data, test_data = data[0:train_size,:], data[train_size:len(data),:]

        # Reshape the data into X=t and Y=t+1
        look_back = 3  # Increase the look-back window
        X_train, Y_train = create_dataset(train_data, look_back)
        X_test, Y_test = create_dataset(test_data, look_back)
        # Reshape the input to be [samples, time steps, features]
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        # Create and fit the RNN network
        model = Sequential()
        model.add(GRU(32, input_shape=(1, look_back), return_sequences=True))  # Use GRU layer with 32 units and return_sequences=True for stacking
        model.add(GRU(16))  # Add another GRU layer with 16 units
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='Nadam')  # Use Nadam optimizer
        model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

        # Make predictions
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Invert predictions
        train_predict = scaler.inverse_transform(train_predict)
        Y_train = scaler.inverse_transform([Y_train])
        test_predict = scaler.inverse_transform(test_predict)
        Y_test = scaler.inverse_transform([Y_test])

        # Evaluate the model
        mae = mean_absolute_error(Y_test[0], test_predict[:,0])
        mse = mean_squared_error(Y_test[0], test_predict[:,0])
        rmse = np.sqrt(mse)
        mbe = np.mean(Y_test[0] - test_predict[:,0])
        mape = mean_absolute_percentage_error(Y_test[0], test_predict[:,0])

        print(f"RMSE: {rmse:.2f}")
        writer.writerow(['RMSE', rmse])
        print(f"MAE: {mae:.2f}")
        writer.writerow(['MAE', mae])
        print(f"MBE: {mbe:.2f}")
        writer.writerow(['MBE', mbe])
        print(f"MAPE: {mape:.2f}%")
        writer.writerow(['MAPE', mape])

        # Plot the results
        plt.figure(figsize=(14, 6))
        plt.plot(data, label='Observed')
        plt.plot(np.concatenate((train_predict, test_predict)), color='r', label='Predicted')
        plt.xlabel('Date')
        plt.ylabel(tag)
        plt.legend()
        plt.show()

        plt.figure(figsize=(14, 6))
        plt.plot(train_data, label='Train Data', color='blue')
        plt.plot(np.arange(len(train_data), len(train_data) + len(test_data)), test_data, label='Test Data', color='green')
        plt.plot(np.concatenate((train_predict, test_predict)), color='red', linestyle='--', label='Predicted')
        plt.xlabel('Date')
        plt.ylabel(tag)
        plt.legend()
        plt.show()

